# НИР. Множественное выравнивание границ экзон-интрон и интрон-экзон

## 1. Поиск границ экзон-интрон и интрон-экзон

In [1]:
import os
import re

#### Загрузка хромосомы и выделение регионов:

In [2]:
path = 'C:/Users/kshch/Projects/УИР/src/data'

In [3]:
# Homo_sapiens.GRCh38.dna.chromosome.1.fa

os.chdir(r"{}".format(path))
files = os.listdir()
files

['borders',
 'Homo_sapiens.GRCh38.107.chromosome.1.gff3',
 'Homo_sapiens.GRCh38.dna.chromosome.1.fa',
 'methods',
 'searching']

In [4]:
with open('Homo_sapiens.GRCh38.107.chromosome.1.gff3', 'r') as file:
    regions = ''.join(file.readlines()).split('###\n')[1:]

#### Выделение среди регионов участков ген:

In [5]:
genes = []

for region in regions:
    if '\tgene\t' in region:
        genes.append(region)

In [6]:
for i in range(len(genes)):
    genes[i] = re.sub('[\tgene\t].+biological_region.+\\n', '', genes[i], flags=0) 

In [7]:
len(genes)

2097

In [8]:
# genes_m = []
# genes_p = []

# for i in range(len(genes)):
#     symb = set()
#     line = genes[i].split('\n')[0]
#     symb.add(line.split('\t')[6])
    
#     if '+' in symb:      
#         genes_p.append(genes[i])
#     elif '-' in symb:
#         genes_m.append(genes[i])

In [9]:
# len(genes_m)

In [10]:
# len(genes_p)

In [11]:
for gene in genes[:10]:
    print(gene)
    print('--'*50)

1	ensembl_havana	gene	65419	71585	.	+	.	ID=gene:ENSG00000186092;Name=OR4F5;biotype=protein_coding;description=olfactory receptor family 4 subfamily F member 5 [Source:HGNC Symbol%3BAcc:HGNC:14825];gene_id=ENSG00000186092;logic_name=ensembl_havana_gene_homo_sapiens;version=7
1	havana	mRNA	65419	71585	.	+	.	ID=transcript:ENST00000641515;Parent=gene:ENSG00000186092;Name=OR4F5-201;biotype=protein_coding;tag=basic;transcript_id=ENST00000641515;version=2
1	havana	exon	65419	65433	.	+	.	Parent=transcript:ENST00000641515;Name=ENSE00003812156;constitutive=1;ensembl_end_phase=-1;ensembl_phase=-1;exon_id=ENSE00003812156;rank=1;version=1
1	havana	five_prime_UTR	65419	65433	.	+	.	Parent=transcript:ENST00000641515
1	havana	five_prime_UTR	65520	65564	.	+	.	Parent=transcript:ENST00000641515
1	havana	exon	65520	65573	.	+	.	Parent=transcript:ENST00000641515;Name=ENSE00003813641;constitutive=1;ensembl_end_phase=0;ensembl_phase=-1;exon_id=ENSE00003813641;rank=2;version=1
1	havana	CDS	65565	65573	.	+	0	ID=

#### Выделение мРНК, кодирующих белок:

In [12]:
mRNA_list = []

for gen in genes:
    i_mRNAs = re.split('\\n.+mRNA', gen)[1:]
    
    if len(i_mRNAs) > 1:
        mRNA_list += i_mRNAs

In [13]:
mRNA_list_new = []

for i in range(len(mRNA_list)):
    if 'protein_coding' in mRNA_list[i].split('\n')[0]:
        mRNA_list[i] = '\n'.join(mRNA_list[i].split('\n')[1:])
        mRNA_list_new.append(mRNA_list[i])

In [14]:
mRNA_list = mRNA_list_new

#### Поиск иных участков:

In [15]:
len(mRNA_list)

7577

In [16]:
names = set()

for mRNA in mRNA_list:
    lines = mRNA.split('\n')
    
    for line in lines:
        if line != '':
            names.add(line.split('\t')[2])

In [17]:
names

{'CDS',
 'exon',
 'five_prime_UTR',
 'lnc_RNA',
 'three_prime_UTR',
 'unconfirmed_transcript'}

In [18]:
for i in range(len(mRNA_list)):
    mRNA_list[i] = re.sub('(\\n.+lnc_RNA(.|\s)*|\\n.+unconfirmed_transcript(.|\s)*)', '', mRNA_list[i], flags=0) 

In [19]:
names = set()

for mRNA in mRNA_list:
    lines = mRNA.split('\n')
    
    for line in lines:
        if line != '':
            names.add(line.split('\t')[2])

In [20]:
names

{'CDS', 'exon', 'five_prime_UTR', 'three_prime_UTR'}

In [21]:
len(mRNA_list)

7577

#### Выбираем границы экзон-интрон и интрон-экзон:

In [22]:
for i in range(len(mRNA_list)):
    symb = set()
    line = mRNA_list[i].split('\n')[0]
    symb.add(line.split('\t')[6])
    
    count_h = mRNA_list[i].count('five_prime_UTR')
    count_t = mRNA_list[i].count('three_prime_UTR')
    
    if '+' in symb:      
        mRNA_list[i] = ''
#         for j in range(count_h - 1):
#             mRNA_list[i] = '\n'.join(mRNA_list[i].split('\n')[2:])
    
#         for j in range(count_t - 1):
#             mRNA_list[i] = '\n'.join(mRNA_list[i].split('\n')[:-2])
    elif '-' in symb:
#         mRNA_list[i] = ''
        for j in range(count_h - 1):
            mRNA_list[i] = '\n'.join(mRNA_list[i].split('\n')[:-2])
    
        for j in range(count_t - 1):
            mRNA_list[i] = '\n'.join(mRNA_list[i].split('\n')[2:])


#### Получение кодирующих участков:

In [23]:
mRNA_list = list(filter(lambda item: item != '', mRNA_list))

In [24]:
len(mRNA_list)

3541

In [25]:
mRNAs_cds_list = []

for mRNA in mRNA_list:
    i_mRNA_cds = []
    
    lines = mRNA.split('\n')
    for line in lines:
        if '\tCDS\t' in line:
            values = line.split('\t')
            CDS = {'start': int(values[3]),
                  'end':    int(values[4])}
            i_mRNA_cds.append(CDS)
    
    mRNAs_cds_list.append(i_mRNA_cds)

In [26]:
mRNAs_cds_list[-3]

[{'start': 248850210, 'end': 248850516},
 {'start': 248850682, 'end': 248850781},
 {'start': 248853937, 'end': 248854051},
 {'start': 248855380, 'end': 248855458},
 {'start': 248855725, 'end': 248855943}]

#### Выделение границ:

In [27]:
with open('Homo_sapiens.GRCh38.dna.chromosome.1.fa', 'r') as file:
    lines = ''.join(''.join(file.readlines()[1:]).split('\n'))

len(lines)

248956422

In [28]:
# границы
exon_intron = []
intron_exon = []

for mRNA_cds in mRNAs_cds_list:    
    for cds in mRNA_cds[:-1]:
        index = cds['end'] - 1
        exon_intron.append((lines[index - 49 : index + 51], cds['end']))

    for cds in mRNA_cds[1:]:
        index = cds['start'] -1
        intron_exon.append((lines[index - 50 : index + 50], cds['start']))

In [29]:
intron_exon

[('CCTCGGTCTGGCTTCTAGGCGCAGCTCCCTCTGCCCCCAGGCACCCAAACCTGTTTTCCAGGCGTCTGGGGTATGCGGATCTGACGTCCTCACAGCCCAG',
  976499),
 ('ACAGTGTGTGCCTGGGATCTGGACGGGCTGTGGGATCCGAGAGCACGTACCTGCCCGTCTAAGAGCCAGGTGGAGCCGCTCTACCACGGCTGGCTTGAGG',
  978881),
 ('CCTCGGTCTGGCTTCTAGGCGCAGCTCCCTCTGCCCCCAGGCACCCAAACCTGTTTTCCAGGCGTCTGGGGTATGCGGATCTGACGTCCTCACAGCCCAG',
  976499),
 ('ACAGTGTGTGCCTGGGATCTGGACGGGCTGTGGGATCCGAGAGCACGTACCTGCCCGTCTAAGAGCCAGGTGGAGCCGCTCTACCACGGCTGGCTTGAGG',
  978881),
 ('GTTCAGGCCCCGGGCACCACCCTGCACATGCCGCCCCTGCCCCACGAGACCTGGGAGCTCCCACCGCCTCGGGGCTCCATCCATGCCCTGAAAGGAAGAG',
  981137),
 ('CCTCGGTCTGGCTTCTAGGCGCAGCTCCCTCTGCCCCCAGGCACCCAAACCTGTTTTCCAGGCGTCTGGGGTATGCGGATCTGACGTCCTCACAGCCCAG',
  976499),
 ('ACAGTGTGTGCCTGGGATCTGGACGGGCTGTGGGATCCGAGAGCACGTACCTGCCCGTCTAAGAGCCAGGTGGAGCCGCTCTACCACGGCTGGCTTGAGG',
  978881),
 ('GGGTCCCGCGCCCTCCCCCCGCCTCCAAGCCGCCGCCGCCCGCGCCTCACCCGTCACCTGCACGCGACGCAGGCTCCGCAGGTGTCTCACGGTCATCTCC',
  999526),
 ('GGTCAGCTGCGACCCAGACTCCGGGTCTCGGGCCTTCGCCCCCGACTTACCTCTTTTCTGA

In [30]:
exon_intron

[('CCGGCGGAAGTAGCGGATGGCAGAGATGGTGCCGACGTTGGCCAGCAAGGCTGCAAGAGAAGCACAGGCTCTTCTGAGGGCCAGCCTGGCTGTCGGCACC',
  976269),
 ('CATGTCATTCTGGCTGAAGGCAAATGATGGGACAGGCCCCCGGAGCTCCCCTAGGACAGAAGCTCACCTTCAGCCCCACGGCTGCACTCAGAGATGGCCC',
  976624),
 ('CCGGCGGAAGTAGCGGATGGCAGAGATGGTGCCGACGTTGGCCAGCAAGGCTGCAAGAGAAGCACAGGCTCTTCTGAGGGCCAGCCTGGCTGTCGGCACC',
  976269),
 ('CATGTCATTCTGGCTGAAGGCAAATGATGGGACAGGCCCCCGGAGCTCCCCTAGGACAGAAGCTCACCTTCAGCCCCACGGCTGCACTCAGAGATGGCCC',
  976624),
 ('AGCCTCTGCATCTGGTCTCCAGAAGACGCCGGACCAGGGCAGGATGAGAACTGGCTGGGAGGGCTGGCGCCGGGGCCGAGGGACGGTGGAGCTTCTGCCC',
  980657),
 ('CCGGCGGAAGTAGCGGATGGCAGAGATGGTGCCGACGTTGGCCAGCAAGGCTGCAAGAGAAGCACAGGCTCTTCTGAGGGCCAGCCTGGCTGTCGGCACC',
  976269),
 ('CATGTCATTCTGGCTGAAGGCAAATGATGGGACAGGCCCCCGGAGCTCCCCTAGGACAGAAGCTCACCTTCAGCCCCACGGCTGCACTCAGAGATGGCCC',
  976624),
 ('GAAGCCGGCGCGGTACTTGCCCAGAACGGCGGGGTCGGCGCTGAGCGCGGCTGCGGGAGCGACACAGGAGGAGAGGTCGGTGCCGGGTCCCGGGGGTCCC',
  999432),
 ('ACGGTCATCTCCAGGATGTCCGCCTTCTCCAGCTTCGAGTGGCGGGAGCTCTGGGGGCGGG

In [31]:
len(intron_exon)

29250

In [32]:
len(exon_intron)

29250

#### Сохранение в файл:

In [33]:
path

'C:/Users/kshch/Projects/УИР/src/data'

In [34]:
with open(path + f'/borders/exon_intron.fasta', 'a') as file:
    for i in range(len(exon_intron)):
        file.write(f'>{i}\n')
        file.write(exon_intron[i][0] + '\n\n')
        
with open(path + f'/borders/exon_intron_loc.fasta', 'a') as file:
    for i in range(len(exon_intron)):
        file.write(f'>{i} ' + str(exon_intron[i][1]) + '\n\n')

with open(path + f'/borders/intron_exon.fasta', 'a') as file:
    for i in range(len(intron_exon)):
        file.write(f'>{i}\n')
        file.write(intron_exon[i][0] + '\n\n')

with open(path + f'/borders/intron_exon_loc.fasta', 'a') as file:
    for i in range(len(intron_exon)):
        file.write(f'>{i} ' + str(intron_exon[i][1]) + '\n\n')

#### Получение выборки

In [35]:
import random

In [36]:
exon_intron_rdm = []
intron_exon_rdm = []
data_size = 200


for i in range(data_size):
    exon_intron_rdm.append(random.choice(exon_intron))
    intron_exon_rdm.append(random.choice(intron_exon))

In [37]:
# exon_intron = '\n\n'.join(exon_intron)
# intron_exon = '\n\n'.join(intron_exon)

with open(path + f'/borders/exon_intron_{data_size}.fasta', 'a') as file:
    for i in range(len(exon_intron_rdm)):
        file.write(f'>{i}\n')
        file.write(exon_intron_rdm[i][0] + '\n\n')
        
with open(path + f'/borders/exon_intron_loc_{data_size}.fasta', 'a') as file:
    for i in range(len(exon_intron_rdm)):
        file.write(f'>{i} ' + str(exon_intron_rdm[i][1]) + '\n\n')

with open(path + f'/borders/intron_exon_{data_size}.fasta', 'a') as file:
    for i in range(len(intron_exon_rdm)):
        file.write(f'>{i}\n')
        file.write(intron_exon_rdm[i][0] + '\n\n')
        
with open(path + f'/borders/intron_exon_loc_{data_size}.fasta', 'a') as file:
    for i in range(len(intron_exon_rdm)):
        file.write(f'>{i} ' + str(intron_exon_rdm[i][1]) + '\n\n')

#### 